In [ ]:
import math

import geopandas as gpd
import numpy as np
import pandas as pd
from dask.dataframe.methods import values
from shapely import LineString, MultiPolygon, Point, Polygon, GeometryCollection
from shapely.ops import polygonize, unary_union



In [ ]:
import pandas as pd

data = {
    30: {
        63: 0,
        125: 0.0002,
        250: 0.0009,
        500: 0.003,
        1000: 0.0075,
        2000: 0.014,
        4000: 0.025,
        8000: 0.064
    },
    20: {
        63: 0,
        125: 0.0003,
        250: 0.0011,
        500: 0.0028,
        1000: 0.0052,
        2000: 0.0096,
        4000: 0.025,
        8000: 0.083
    },
    10: {
        63: 0,
        125: 0.0004,
        250: 0.001,
        500: 0.002,
        1000: 0.0039,
        2000: 0.01,
        4000: 0.035,
        8000: 0.125
    },
    0: {
        63: 0,
        125: 0.0004,
        250: 0.0008,
        500: 0.0017,
        1000: 0.0049,
        2000: 0.017,
        4000: 0.058,
        8000: 0.156
    }
}

air_resist_ratio = pd.DataFrame(data)


def get_nearest_values(array, value):
    sorted_array = sorted(array)
    if value in sorted_array:
        return [value]
    if value > max(sorted_array):
        return [sorted_array[-1]]
    if value < min(sorted_array):
        return [sorted_array[0]]

    for i, val in enumerate(sorted_array):
        if value < val:
            return sorted_array[max(i - 1, 0)], sorted_array[i]
    return sorted_array[-2], sorted_array[-1]


def get_air_resist_ratio(temp, freq):
    nearest_temp = get_nearest_values(air_resist_ratio.columns, temp)
    nearest_freq = get_nearest_values(air_resist_ratio.index, freq)

    if len(nearest_temp) == 1 and len(nearest_freq) == 1:
        return air_resist_ratio.loc[nearest_freq[0], nearest_temp[0]]

    if len(nearest_temp) == 2 and len(nearest_freq) == 2:
        freq1, freq2 = nearest_freq
        temp1, temp2 = nearest_temp

        coef_temp1_freq1 = air_resist_ratio.loc[freq1, temp1]
        coef_temp1_freq2 = air_resist_ratio.loc[freq2, temp1]
        coef_temp2_freq1 = air_resist_ratio.loc[freq1, temp2]
        coef_temp2_freq2 = air_resist_ratio.loc[freq2, temp2]

        weight_temp1 = (temp2 - temp) / (temp2 - temp1)
        weight_temp2 = (temp - temp1) / (temp2 - temp1)
        weight_freq1 = (freq2 - freq) / (freq2 - freq1)
        weight_freq2 = (freq - freq1) / (freq2 - freq1)

        coef_freq1 = coef_temp1_freq1 * weight_temp1 + coef_temp2_freq1 * weight_temp2
        coef_freq2 = coef_temp1_freq2 * weight_temp1 + coef_temp2_freq2 * weight_temp2

        final_coef = coef_freq1 * weight_freq1 + coef_freq2 * weight_freq2

        return final_coef

    if len(nearest_temp) == 2 and len(nearest_freq) == 1:
        temp1, temp2 = nearest_temp
        freq1 = nearest_freq[0]

        coef_temp1 = air_resist_ratio.loc[freq1, temp1]
        coef_temp2 = air_resist_ratio.loc[freq1, temp2]

        weight_temp1 = (temp2 - temp) / (temp2 - temp1)
        weight_temp2 = (temp - temp1) / (temp2 - temp1)

        return coef_temp1 * weight_temp1 + coef_temp2 * weight_temp2

    if len(nearest_temp) == 1 and len(nearest_freq) == 2:
        temp1 = nearest_temp[0]
        freq1, freq2 = nearest_freq

        coef_freq1 = air_resist_ratio.loc[freq1, temp1]
        coef_freq2 = air_resist_ratio.loc[freq2, temp1]

        weight_freq1 = (freq2 - freq) / (freq2 - freq1)
        weight_freq2 = (freq - freq1) / (freq2 - freq1)

        return coef_freq1 * weight_freq1 + coef_freq2 * weight_freq2


In [ ]:
import numpy as np
from scipy.optimize import fsolve


def dist_to_target_db(init_noise_db, target_noise_db, geometric_mean_freq_hz, air_temperature, return_desc=False):
    def equation(r):
        return L - L_ist + 20 * np.log10(r) + k * r

    L_ist = init_noise_db
    L = target_noise_db
    k = get_air_resist_ratio(air_temperature, geometric_mean_freq_hz)
    initial_guess = 1
    r_solution = fsolve(equation, initial_guess)
    if return_desc:
        string = f"Шум громкостью {init_noise_db} дБ,среднегеометрической частотой {geometric_mean_freq_hz} Гц при температуре {air_temperature} Цельсия затухает к шуму {target_noise_db} дБ за расстояние {r_solution} \nКоэффициент сопротивления воздуха {k}"
        return r_solution[0], string
    return r_solution[0]


def green_noise_reduce_db(geometric_mean_freq_hz, r_tree):
    return round(0.08 * r_tree * ((geometric_mean_freq_hz ** (1 / 3)) / 8), 1)


dist_to_target_db(100, 40, 1000, 20)

In [ ]:
from objectnat.methods.utils.geom_utils import polygons_to_multilinestring
from shapely.ops import unary_union, polygonize


def gdf_to_circle_zones_from_point(gdf: gpd.GeoDataFrame, point_from: Point, zone_radius,
                                   resolution=4) -> gpd.GeoDataFrame:
    """ n_segments = 4*resolution,e.g. if resolution = 4 that means there will be 16 segments"""
    crs = gdf.crs
    buffer = point_from.buffer(zone_radius, resolution=resolution)
    buffer0_1 = point_from.buffer(0.1, resolution=resolution)
    gdf_geometry = gdf.clip(buffer, keep_geom_type=True).geometry.apply(polygons_to_multilinestring).unary_union
    zones_lines = [LineString([Point(coords1), Point(coords2)]) for coords1, coords2 in
                   zip(buffer.exterior.coords, buffer0_1.exterior.coords)]
    return gpd.GeoDataFrame(geometry=list(polygonize(unary_union([gdf_geometry] + zones_lines))), crs=crs)

In [141]:

%reload_ext autoreload
%autoreload 2

import geopandas as gpd
from objectnat import get_visibility_accurate
import concurrent.futures
import multiprocessing
import time
from objectnat.methods.utils.geom_utils import polygons_to_multilinestring, get_point_from_a_thorough_b
from shapely.geometry import LineString, MultiPolygon, Point, Polygon
from shapely.ops import unary_union

total_res = []
visited_points = []
ban_area = []

wide_points = []
kek_points = []
polygons_z = []


def noise_from_point_task(task, **kwargs) -> tuple[gpd.GeoDataFrame, list[tuple] | None]:
    # Unpacking task
    point_from, obstacles, trees, passed_dist, deep, dist_db = task

    def donuts_dist_values(dist_db, passed_dist, max_view_dist):
        new_dist_db = dist_db + [(passed_dist, None), (max_view_dist + passed_dist, None)]
        new_dist_db = sorted(new_dist_db, key=lambda x: x[0])
        start = None
        end = None
        for i, (dist, db) in enumerate(new_dist_db):
            if db is None:
                if start is None:
                    new_dist_db[i] = (dist, new_dist_db[i - 1][1])
                    start = i
                else:
                    new_dist_db[i] = (dist, new_dist_db[i + 1][1])
                    end = i + 1
                    break
        return [(dist - passed_dist, db) for dist, db in new_dist_db[start:end]]

    max_dist = max(dist_db, key=lambda x: x[0])[0]
    reflection_n = kwargs.get('reflection_n')
    geometric_mean_freq_hz = kwargs.get('geometric_mean_freq_hz')
    local_crs = obstacles.crs
    dist = round(max_dist - passed_dist, 1)

    obstacles = obstacles[obstacles.intersects(point_from.buffer(dist, resolution=8))]

    if len(obstacles) == 0:
        obstacles_union = Polygon()
    else:
        obstacles_union = obstacles.unary_union

    vis_poly, max_view_dist = get_visibility_accurate(point_from, obstacles, dist, return_max_view_dist=True)
    donuts_dist_values = donuts_dist_values(dist_db, passed_dist, max_view_dist)

    min_noise = donuts_dist_values[-1][1]
    print('min_noise: ', min_noise)
    total_res.append(vis_poly)
    visited_points.append(point_from)
    allowed_geom_types = ["MultiPolygon", "Polygon"]
    # Trees noise reduce
    to_remove = []
    tree_intersection = trees.intersects(vis_poly)
    if tree_intersection.any():
        for ind, row in trees[tree_intersection].iterrows():
            tree_geom = row.geometry
            points_with_angle = [(Point(pt), round(math.atan2(pt[1] - point_from.y, pt[0] - point_from.x),5), Point(pt).distance(point_from)) for pt in tree_geom.exterior.coords]
            p0_1 = max(points_with_angle, key=lambda x: (x[1],x[2]))
            p0_2 = min(points_with_angle, key=lambda x: (x[1],-x[2]))
            
            delta_angle = 2 * math.pi + p0_1[1] - p0_2[1]

            if delta_angle > math.pi:
                delta_angle = 2 * math.pi - delta_angle
            a = math.sqrt((dist ** 2) * (1 + (math.tan(delta_angle / 2) ** 2)))
            p1 = get_point_from_a_thorough_b(point_from, p0_1[0], a)
            p2 = get_point_from_a_thorough_b(point_from, p0_2[0], a)
            polygon = Polygon([p0_1[0], p1, p2, p0_2[0]])
            to_remove = to_remove + [tree_geom, polygon]
            donuts = []
            don_values = []
            to_cut_off = point_from
            r_tree = round(tree_geom.area / p0_1[0].distance(p0_2[0]), 1)

            noise_reduce = green_noise_reduce_db(geometric_mean_freq_hz, r_tree)

            for i in range(len(donuts_dist_values[:-1])):
                don_value = donuts_dist_values[i][1] - noise_reduce
                cur_buffer = point_from.buffer(donuts_dist_values[i + 1][0])
                if don_value >= min_noise:
                    donuts.append(cur_buffer.difference(to_cut_off))
                    don_values.append(don_value)
                    to_cut_off = cur_buffer
                else:
                    break
            noise_after_tree = gpd.GeoDataFrame(geometry=donuts, data={'noise_level': don_values}, crs=local_crs).clip(
                polygon.difference(tree_geom), keep_geom_type=True).explode(ignore_index=True)
            noise_after_tree = noise_after_tree[noise_after_tree.geom_type.isin(allowed_geom_types)]
            wide_points.append(noise_after_tree)

    vis_poly = vis_poly.difference(unary_union(to_remove))
    
    donuts = []
    don_values = []
    to_cut_off = point_from

    for i in range(len(donuts_dist_values[:-1])):
        cur_buffer = point_from.buffer(donuts_dist_values[i + 1][0])
        donuts.append(cur_buffer.difference(to_cut_off))
        don_values.append(donuts_dist_values[i][1])
        to_cut_off = cur_buffer

    noise_from_point = gpd.GeoDataFrame(geometry=donuts, data={'noise_level': don_values}, crs=local_crs).clip(
        vis_poly, keep_geom_type=True).explode(
        ignore_index=True)

    noise_from_point = noise_from_point[noise_from_point.geom_type.isin(allowed_geom_types)]

    if deep == reflection_n:
        return noise_from_point, None
    print(type(vis_poly))
    if isinstance(vis_poly, Polygon):
        vis_poly_points = [Point(coords) for coords in vis_poly.exterior.coords]
    elif isinstance(vis_poly, MultiPolygon):
        vis_poly_points = [Point(coords) for geom in vis_poly.geoms for coords in geom.exterior.coords]
    vis_poly_points = gpd.GeoDataFrame(geometry=vis_poly_points,
                                       crs=local_crs)

    nearby_poly = point_from.buffer(1.1, resolution=2)
    # Generating reflection points
    vis_poly_points['point'] = vis_poly_points.geometry
    vis_poly_points.geometry = vis_poly_points.geometry.buffer(1, resolution=1)
    vis_poly_points = vis_poly_points.sjoin(obstacles, predicate='intersects')
    vis_poly_points.dropna(subset=['absorb_ratio'], inplace=True)
    vis_poly_points.geometry = vis_poly_points.difference(vis_poly).difference(obstacles_union).difference(nearby_poly)
    vis_poly_points = vis_poly_points[~vis_poly_points.is_empty]
    vis_poly_points = vis_poly_points[vis_poly_points.area >= 0.1]
    vis_poly_points.geometry = vis_poly_points['point']
    vis_poly_points['dist'] = vis_poly_points.distance(point_from)
    vis_poly_points = vis_poly_points[vis_poly_points['dist'] < max_dist - 5]

    if len(vis_poly_points) == 0:
        return noise_from_point, None

    new_obs = pd.concat([obstacles, gpd.GeoDataFrame(geometry=[vis_poly], crs=local_crs)], ignore_index=True)

    new_tasks = []
    for _, loc in vis_poly_points.iterrows():
        new_passed_dist = loc.dist + passed_dist
        dist_last = max_dist - new_passed_dist
        if dist_last > 1:
            dist_change = loc['absorb_ratio'] * dist_last
            new_dist_db = [(dist - dist_change, db) for dist, db in dist_db]
            task_obs = new_obs.copy()
            task_obs.geometry = task_obs.difference(loc.geometry.buffer(1, resolution=1))
            new_tasks.append(
                (noise_from_point_task, (loc.geometry, task_obs, new_passed_dist, deep + 1, new_dist_db), kwargs))

    return noise_from_point, new_tasks


def parallel_split_queue(task_queue: multiprocessing.Queue, dead_area: Polygon, dead_area_r: int):
    results = []
    # with concurrent.futures.ProcessPoolExecutor() as executor:
    with concurrent.futures.ThreadPoolExecutor(max_workers=1) as executor:
        future_to_task = {}
        while True:
            while not task_queue.empty() and len(future_to_task) < executor._max_workers:
                func, task, kwargs = task_queue.get_nowait()
                future = executor.submit(func, task, **kwargs)
                future_to_task[future] = task
            done, _ = concurrent.futures.wait(future_to_task.keys(), return_when=concurrent.futures.FIRST_COMPLETED)
            for future in done:
                future_to_task.pop(future)
                result, new_tasks = future.result()
                if new_tasks:
                    new_dead_area_points = [dead_area]
                    for func, new_task, kwargs in new_tasks:
                        if not dead_area.covers(new_task[0]):
                            task_queue.put((func, new_task, kwargs))
                            new_dead_area_points.append(new_task[0].buffer(dead_area_r, resolution=2))
                    dead_area = unary_union(new_dead_area_points)
                results.append(result)
            time.sleep(0.01)
            if not future_to_task and task_queue.empty():
                break
    ban_area.append(dead_area)
    return results


def simulate_noise(point_from: Point, obstacles: gpd.GeoDataFrame, trees=None, dB_step=5, reflection_n=3, **kwargs):
    obstacles = obstacles.copy()
    trees = trees.copy()

    if len(obstacles) > 0:
        local_crs = obstacles.estimate_utm_crs()
        obstacles.to_crs(local_crs, inplace=True)

    else:
        local_crs = obstacles.crs

    trees.to_crs(local_crs, inplace=True)

    init_noise_db = kwargs['init_noise_db']
    target_noise_db = kwargs['target_noise_db']
    geometric_mean_freq_hz = kwargs['geometric_mean_freq_hz']
    air_temperature = kwargs['air_temperature']
    dead_area_r = kwargs.get('dead_area_r', 5)
    standart_absorb_ratio = kwargs.get('standart_absorb_ratio', 0.5)

    absorb_ratio_column = kwargs.get('absorb_ratio_column', None)
    if absorb_ratio_column is None:
        obstacles['absorb_ratio'] = standart_absorb_ratio
    else:
        obstacles['absorb_ratio'] = obstacles[absorb_ratio_column]
        obstacles['absorb_ratio'] = obstacles['absorb_ratio'].fillna(standart_absorb_ratio)
    obstacles = obstacles[['absorb_ratio', 'geometry']]

    dist_db = [(0, init_noise_db)]
    cur_dB = init_noise_db - dB_step
    max_dist = 0
    while cur_dB != target_noise_db - dB_step:
        max_dist = dist_to_target_db(init_noise_db, cur_dB, geometric_mean_freq_hz, air_temperature)
        dist_db.append((max_dist, cur_dB))
        cur_dB = cur_dB - dB_step

    trees = gdf_to_circle_zones_from_point(trees, point_from, max_dist, resolution=4)
    trees.geometry = trees.geometry.simplify(tolerance=1)

    # creating initial task
    task_queue = multiprocessing.Queue()
    args = (point_from, obstacles, trees, 0, 0, dist_db)
    kwargs = {'reflection_n': reflection_n, 'geometric_mean_freq_hz': geometric_mean_freq_hz}
    task_queue.put((noise_from_point_task, args, kwargs))
    noise_gdf = parallel_split_queue(task_queue, dead_area=point_from.buffer(dead_area_r, resolution=2),
                                     dead_area_r=dead_area_r)
    print("done")
    noise_gdf = pd.concat(noise_gdf, ignore_index=True)
    polygons = gpd.GeoDataFrame(
        geometry=list(polygonize(noise_gdf.geometry.apply(polygons_to_multilinestring).unary_union)), crs=local_crs)
    polygons_points = polygons.copy()
    polygons_points.geometry = polygons.representative_point()
    joined = polygons_points.sjoin(noise_gdf, predicate="within").reset_index()
    joined = joined.groupby("index").agg({"noise_level": 'max'})
    joined['geometry'] = polygons
    joined = gpd.GeoDataFrame(joined, geometry="geometry", crs=local_crs).dissolve(by='noise_level').reset_index()
    return joined


start_p = Point(347258.80, 6648128.41)
obstacles = gpd.read_file("Здания,Территория _Васильевский_.geojson").to_crs(32636)
obstacles = gpd.GeoDataFrame(geometry=[], crs=32636)
trees = gpd.read_file('2tree.geojson').to_crs(32636)
# obstacles = gpd.read_parquet('buildings.parquet').to_crs(epsg=32636)
noise = simulate_noise(start_p,
                       obstacles,
                       trees,
                       dB_step=2.5,
                       init_noise_db=95,
                       target_noise_db=40,
                       geometric_mean_freq_hz=2000,
                       air_temperature=20,
                       reflection_n=3,
                       dead_area_r=5
                       )

min_noise:  40.0
<class 'shapely.geometry.multipolygon.MultiPolygon'>
done


In [142]:
m1 = noise.explore(column='noise_level', cmap='RdYlGn_r', vmin=0, vmax=140,
                   style_kwds={'fillOpacity': 0.4})
trees.explore(m=m1, color='green')
pd.concat(wide_points, ignore_index=True).explore(m=m1, column='noise_level', cmap='RdYlGn_r', vmin=0, vmax=140,
                                                  style_kwds={'fillOpacity': 0.4})
# gpd.GeoDataFrame(geometry=polygons_z[0], crs=32636).explore(m=m1, color='purple')

# gpd.GeoDataFrame(geometry=trees.centroid,crs=32636).explore(m=m1,color='red')

In [ ]:
m1 = obstacles.explore(color='pink', tiles='CartoDB positron')
noise.reset_index().explore(m=m1, column='noise_level', cmap='RdYlGn_r', vmin=0, vmax=140,
                            style_kwds={'fillOpacity': 0.4})
gpd.GeoDataFrame(geometry=visited_points, crs=32636).reset_index().explore(m=m1, column='index')

In [ ]:
m1 = gpd.GeoDataFrame(geometry=total_res, crs=32636).explore()
gpd.GeoDataFrame(geometry=ban_area, crs=32636).explore(m=m1, color='red')
gpd.GeoDataFrame(geometry=visited_points, crs=32636).reset_index().explore(m=m1, column='index')


In [ ]:
ban_area

In [ ]:
m1 = obstacles.explore(color='pink', tiles='CartoDB positron')
initial_sim.explore(color='purple', m=m1)
gpd.GeoDataFrame(geometry=[vis_poly], crs=32636).explore(m=m1)
m1

In [ ]:
%reload_ext autoreload
%autoreload 2

new_obs = pd.concat([obstacles, gpd.GeoDataFrame(geometry=[res], crs=32636)])
p = [Point(coords) for coords in res.simplify(1).exterior.coords][14]
new_obs.geometry = new_obs.geometry.difference(p.buffer(1, quad_segs=1))
vis_poly2 = get_visibility_accurate(p, new_obs, 500)
# m1=new_obs.explore(color='red')
# m1= gpd.GeoDataFrame(geometry=[vis_poly],crs=32636).explore()
# gpd.GeoDataFrame(geometry=[Point((347810.97481683653,6647261.617306325))],crs=32636).explore(m=m1)

In [ ]:
gpd.GeoDataFrame(geometry=vis_poly, crs=32636)
m1 = res.explore(column='noise_dB', cmap='RdYlGn_r', vmin=10, tiles='CartoDB positron')
obstacles.explore(m=m1, color='pink')
m1
m1 = obstacles.explore(color='pink')
gpd.GeoDataFrame(geometry=[result2], crs=32636).explore(m=m1)
gpd.GeoDataFrame(geometry=[Point(coords) for coords in result2.exterior.coords], crs=32636).explore(m=m1, color='red')

points = [Point(coords) for coords in result2.simplify(1).exterior.coords]
print(len(points))
gpd.GeoDataFrame(geometry=points, crs=32636).explore(m=m1, color='purple')

In [ ]:
result_poly